In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
df_amazon = pd.read_csv('dataset/textual/raw/amazon_products.csv')
df_categories = pd.read_csv('dataset/textual/raw/amazon_categories.csv')

In [3]:
PRODUCTS_PER_CATEGORY = 850

In [4]:
df_amazon.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000


In [5]:
df_amazon.drop(columns=['listPrice'], inplace=True)
df_amazon.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,104,False,1000


In [6]:
category_counts = df_amazon['category_id'].value_counts()
category_counts

category_id
91     28619
84     24660
270    20846
114    19822
118    18994
       ...  
195       76
186       50
185       42
102       40
194       22
Name: count, Length: 248, dtype: int64

In [7]:
# SELECT CATEGORIES WITH MORE THAN (THE DEFINED NUMBER OF) PRODUCTS
valid_categories = category_counts[category_counts > PRODUCTS_PER_CATEGORY].index

# CREATE A LIST OF CATEGORIES. PICK 20 FROM THEM AND SAVE TO CSV - MANUALLY.
# choose_cateogies = pd.DataFrame(valid_categories, columns=['category_id']).merge(df_categories, left_on='category_id', right_on='id', how='left').reset_index(drop=True).drop(columns=['id'])
# choose_cateogies.to_csv('dataset/textual/temp/choose_categories.csv', index=False)

valid_categories = pd.read_csv('dataset/textual/filtered/chosen_categories.csv')
valid_categories.head(2)

,category_id,category_name
0,116,Women's Clothing
1,114,Men's Shoes


In [8]:
# SELECT PRODUCTS FROM THE SELECTED CATEGORIES
selected_products = df_amazon[df_amazon['category_id'].isin(valid_categories['category_id'])].groupby('category_id').apply(lambda x: x.sample(n=PRODUCTS_PER_CATEGORY)).reset_index(drop=True)
selected_products.head(2)

C:\Users\ShahrukhAzharAhsan\AppData\Local\Temp\ipykernel_68644\1394958067.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_products = df_amazon[df_amazon['category_id'].isin(valid_categories['category_id'])].groupby('category_id').apply(lambda x: x.sample(n=PRODUCTS_PER_CATEGORY)).reset_index(drop=True)


,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth
0,B08ZDQX51W,Original Replacement Dell 130W Laptop Charger ...,https://m.media-amazon.com/images/I/61sADwl+YW...,https://www.amazon.com/dp/B08ZDQX51W,4.5,0,24.98,65,False,0
1,B01BPCTXHC,Griffin Elevator Stand for Laptops - Lift Your...,https://m.media-amazon.com/images/I/710N2S69Nv...,https://www.amazon.com/dp/B01BPCTXHC,4.6,0,35.00,65,False,0


In [9]:
selected_products.shape

(10200, 10)

In [10]:
# CHECK THE NUMBER OF CATEGORIES
selected_products.groupby('category_id').size().shape

(12,)

In [11]:
# GET THE CATEGORY NAMES FOR THE SELECTED PRODUCTS
selected_products = selected_products.merge(df_categories, left_on='category_id', right_on='id',how='left').reset_index(drop=True)
selected_products.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth,id,category_name
0,B08ZDQX51W,Original Replacement Dell 130W Laptop Charger ...,https://m.media-amazon.com/images/I/61sADwl+YW...,https://www.amazon.com/dp/B08ZDQX51W,4.5,0,24.98,65,False,0,65,Laptop Accessories
1,B01BPCTXHC,Griffin Elevator Stand for Laptops - Lift Your...,https://m.media-amazon.com/images/I/710N2S69Nv...,https://www.amazon.com/dp/B01BPCTXHC,4.6,0,35.00,65,False,0,65,Laptop Accessories


In [12]:
# CHECK IF THERE ARE EQUAL NUMBER OF PRODUCTS IN EACH CATEGORY
selected_products.groupby('category_name').size()

category_name
Backpacks                       850
Dolls & Accessories             850
Girls' Shoes                    850
Headphones & Earbuds            850
Home Storage & Organization     850
Laptop Accessories              850
Men's Shoes                     850
Sports & Fitness                850
Televisions & Video Products    850
Toys & Games                    850
Women's Clothing                850
Women's Handbags                850
dtype: int64

In [13]:
# Save the final selected products\
selected_products.to_csv('dataset/textual/filtered/selected_products.csv', index=False)

Images

In [14]:
# # SAVE THE IMAGES OF THE SELECTED PRODUCTS (AND RENAME THEM WITH THE ASIN)
# for row in selected_products[['imgUrl', 'asin']].head(100).itertuples():
#     image_link = row.imgUrl
#     # print(image_link)
#     renamed_image = row.asin + ".jpg"
#     print(renamed_image)
#     img_data = requests.get(image_link).content
#     with open('dataset/images/' + renamed_image, 'wb') as handler:
#         handler.write(img_data)